# Assignment # 11  
Park Juyeon, Department of Statistics and Data Science, 2022311137

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from numpy.linalg import inv, det
from statsmodels.stats import multivariate as mv
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix

pd.options.display.float_format = '{:.5f}'.format
np.set_printoptions(suppress=True)
warnings.filterwarnings(action='ignore')
plt.rcParams['figure.facecolor'] = 'white'

----

# Q1. 
### Write a Python code to calculate the linear discriminant function for several classes. Your code should be able to predict the Y class based on the input $x_o$ values.

In [2]:
# Linear discriminant function
def myLDA(X, y, prior, x0):
    """ Multivariate LDA """
    # 1. Dividing groups
    n, p = X.shape
    classes, n_groups = np.unique(y, return_counts=True)
    g = len(classes)  # number of groups

    # 1. Sample mean and sample covariance
    X_bar = np.zeros((g, p))
    S = []
    for i, name in enumerate(classes):
        X_bar[i, :] = X[y == name].mean()
        S.append(X[y == name].cov())

    # 2. Pooled estimate of Sigma
    S_pooled = np.zeros((p, p))
    for i, n_group in enumerate(n_groups):
        S_pooled += (n_group - 1) * S[i] / (n - g)

    # 3. d_i calculation
    ## Coefficient
    coef = [X_bar[i] @ inv(S_pooled) for i in range(g)]
    ## Constant
    const = [X_bar[i] @ inv(S_pooled) @ X_bar[i] * (-1/2)  \
             + np.log(prior[i]) for i in range(g)]
    ## LDF
    d = [coef[i] @ x0 + const[i] for i in range(g)]

    # 4. classification
    pred_idx = np.argmax(d)
    y_pred = classes[pred_idx]
    
    return y_pred, [d, coef, const]

----

# Q2.
### Write a Python code to calculate the quadratic discriminant function for several classes. Your code should be able to predict the Y class based on the input $x_0$ values.

In [3]:
# Linear discriminant function
def myQDA(X, y, prior, x0):
    """ Multivariate QDA """
    # 1. Dividing groups
    n, p = X.shape
    classes, n_groups = np.unique(y, return_counts=True)
    g = len(classes)  # number of groups

    # 1. Sample mean and sample covariance
    X_bar = np.zeros((g, p))
    S = []
    for i, name in enumerate(classes):
        X_bar[i, :] = X[y == name].mean()
        S.append(X[y == name].cov())

    # Pooled estimate of Sigma
    d = [(-1/2) * (np.log(det(S[i])) + (x0-X_bar[i]) @ inv(S[i]) @ (x0-X_bar[i])) \
         + np.log(prior[i]) for i in range(g)]

    # 4. classification
    pred_idx = np.argmax(d)
    y_pred = classes[pred_idx]
    
    return y_pred, d

----

# Q3.
### Write a Python code to perform the ‘leave-one-out’ method to calculate the accuracy of the LDA & QDA model you wrote in #1 and #2.

In [4]:
def prediction(X, y, eval_type, prior, error_type):
    y_pred = np.zeros(len(X))
    for i in range(len(X)):
        # 1. Define target dataset
        if error_type == 'APER':
            new_X = X.copy()  # Use whole dataset to train and test
            new_y = y.copy()
        if error_type == 'LOO':
            new_X = X.drop(i, axis=0)  # leave ith observation to test set
            new_y = y.drop(i, axis=0)
        x0 = X.iloc[i, :]
        
        # 2. LDA
        y_pred_i, _ = eval_type(new_X, new_y, prior, x0)
        y_pred[i] = y_pred_i
    
    return y_pred

# Accuracy
def calculate_accuracy(y, y_pred):
    accuracy = sum(y == y_pred) / len(y)

    return accuracy

----

# Q4. 
### Using Fisher’s Iris data, answer the following questions.


In [5]:
iris = pd.read_csv('iris.dat', delim_whitespace = True, header = None)
Class_name = ['setosa', 'versicolor', 'virginica']
iris.columns = ['Sepal length', 'Sepal width', 'Petal length', 'Petal width', 'Class']  # ["setosa", 'versicolor', 'virginica']
iris

,Sepal length,Sepal width,Petal length,Petal width,Class
0,5.10000,3.50000,1.40000,0.20000,1
1,4.90000,3.00000,1.40000,0.20000,1
2,4.70000,3.20000,1.30000,0.20000,1
3,4.60000,3.10000,1.50000,0.20000,1
4,5.00000,3.60000,1.40000,0.20000,1
...,...,...,...,...,...
145,6.70000,3.00000,5.20000,2.30000,3
146,6.30000,2.50000,5.00000,1.90000,3
147,6.50000,3.00000,5.20000,2.00000,3
148,6.20000,3.40000,5.40000,2.30000,3


In [6]:
X = iris.iloc[:, :-1]
y = iris.iloc[:, -1]

----

## a. Is the assumption of a common covariance matrix reasonable in this case? (Use Python’s statsmodels.stats.multivariate’ module for this question).

In [7]:
## 등분산 검정(homogeneity of variance test) 
# Covariance matrix for each group
classes, n_groups = np.unique(y, return_counts=True)
for i, name in enumerate(classes):
    globals()['cov' + str(i+1)] = X[y == name].cov()

# Testing
test = mv.test_cov_oneway([cov1, cov2, cov3], n_groups)
print(f'Chi-Square Test stat: {test.statistic_chi2 :.5f}, Pr > ChiSq: {test.pvalue_chi2 :.5f}')
print('Reject H0')

Chi-Square Test stat: 140.94305, Pr > ChiSq: 0.00000
Reject H0


From homogenity of variance test, We can conclude that the groups do NOT satisfy homoscedastic assumption. (Reject H0)  
Therefore we should conduct QDA, not LDA.

----

## b. Assuming that the populations are multivariate normal, calculate the quadratic discriminant scores with equal prior and equal misclassification cost. Classify the new observation $x_0 = [5.0, 3.5, 1.75, 0.21]^T$ into one of population $\pi_1, \pi_2, \pi_3$. (Use your code in #2 above).

In [8]:
prior = [1/3, 1/3, 1/3]
x0 = [5.0, 3.5, 1.75, 0.21]

y_pred, score = myQDA(X, y, prior, x0)
score = pd.DataFrame(np.array(score).reshape(1, -1), columns=Class_name)

print(f'The QDA scores are')
print(score, '\n')
print(f'The the prediction is')
Class_name[y_pred - 1]

The QDA scores are
   setosa  versicolor  virginica
0 3.49098   -47.30137  -74.70869 

The the prediction is


'setosa'

The quadratic discriminant scores are [3.49, -47.30, -74.70] for setosa, versicolor and virginica, respectively. Therefore, the new observation x0 can be classified to setosa.

----

## c. Assuming equal covariance matrices and multivariate normal populations, calculate the linear discriminant function using your code in #1 above and compare its coefficient with those of Python’s ‘sklearn.discriminant_analysis’ module.

### LDA with code in #1

In [9]:
prior = [1/3, 1/3, 1/3]
x0 = [5.0, 3.5, 1.75, 0.21]

y_pred, result  = myLDA(X, y, prior, x0)
score, coef, const = result

coef = pd.DataFrame(coef,
                    index=[Class_name], 
                    columns=iris.columns[:-1]).T
const = pd.DataFrame(np.array(const).reshape(1, -1),
                     index=['Const'], 
                     columns=[Class_name])
LDA_info = pd.concat((const, coef), axis=0)

print('The coefficients and constant of my LDA is ')
LDA_info

The coefficients and constant of my LDA is 


,setosa,versicolor,virginica
Const,-86.30847,-72.85261,-104.36832
Sepal length,23.54417,15.69821,12.44585
Sepal width,23.58787,7.07251,3.68528
Petal length,-16.43064,5.21145,12.76654
Petal width,-17.39841,6.43423,21.07911


### LDA with sklearn.discriminant_analysis

#### Fit the LDA in sklearn.discriminant_analysis

In [10]:
lda = LDA(priors=prior)
lda.fit(X, y)

LinearDiscriminantAnalysis(priors=[0.3333333333333333, 0.3333333333333333,
                                   0.3333333333333333])

#### Get coefficients and constant

In [11]:
LDA_module = pd.DataFrame(np.append(lda.intercept_, lda.coef_.T).reshape(5,3),
                          index = ['Constant', 'Sepal length', 'Sepal width', 'Petal length', 'Petal width'],
                          columns = np.sort(y.unique()))
LDA_module.index.names = ['Variable']
LDA_module

,1,2,3
Variable,,,
Constant,-15.47784,-2.02197,-33.53769
Sepal length,6.31476,-1.53120,-4.78356
Sepal width,12.13932,-4.37604,-7.76327
Petal length,-16.94642,4.69567,12.25076
Petal width,-20.77005,3.06259,17.70747


#### Confusion matrix

In [12]:
# Confusion matrix

y_pred = lda.predict(X)
C1 = pd.DataFrame(confusion_matrix(y, y_pred))
C1.index = pd.Series(C1.index).map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})
C1.columns = pd.Series(C1.columns).map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})
C1['Total'] = C1.sum(axis=1)
C1.loc['Total'] = C1.sum(axis=0)

C1

,setosa,versicolor,virginica,Total
setosa,50,0,0,50
versicolor,0,48,2,50
virginica,0,1,49,50
Total,50,49,51,150


Comparing the result of two methods, the coefficients and the constant are different.

----

## d. Calculate and compare the APER and the leave-one-out error rates for linear discriminant analysis (LDA) and quadratic discriminant analysis (QDA) using your code in #1, 2, 3. (Assume equal prior and equal misclassification cost).

In [13]:
y_pred = prediction(X, y, myLDA, prior, 'APER')
accuracy = calculate_accuracy(y, y_pred)
print(f'Accuracy of LDA with APER is {accuracy}')

y_pred = prediction(X, y, myLDA, prior, 'LOO')
accuracy = calculate_accuracy(y, y_pred)
print(f'Accuracy of LDA with LOO is {accuracy}')

y_pred = prediction(X, y, myQDA, prior, 'APER')
accuracy = calculate_accuracy(y, y_pred)
print(f'Accuracy of QDA with APER is {accuracy}')

y_pred = prediction(X, y, myQDA, prior, 'LOO')
accuracy = calculate_accuracy(y, y_pred)
print(f'Accuracy of QDA with LOO is {accuracy}')

Accuracy of LDA with APER is 0.98
Accuracy of LDA with LOO is 0.98
Accuracy of QDA with APER is 0.98
Accuracy of QDA with LOO is 0.9733333333333334
